# TP6 - INF2

Pour ce TP nous allons expliqué la méthode que nous avons suivi.

A savoir :
+ préciser le fichier dans lequel nous ajoutons un élément
+ écrire le code (contenant les explications)

Nous avons également fait le choix de rendre toutes nos vues (url, lié aux vues) disponible sur la base (nous les avons également mis aux endroits adéquats : par exemple dans la page `edit_profile` nous avons les redirections vers `change_password` et `del_user`), afin d'en faciliter la correction. 

En pratique les vues suivantes ne serait pas accessible sur la base :
+ `change_password`
+ `del_user`


**La méthode que nous avons suivi tout au long de ce TP est la suivante :**

Liste des étapes :

+ étape 1 : créé le lien URL, dans `urls.py`
+ étape 2 : créé le formulaire, si nécessaire dans `forms.py`
+ étape 3 : modifié le fichier `models.py`, si nécessaire
+ étape 4 : créé la vue associé dans `views.py`
+ étape 5 : créé la page HTML, si nécessaire `templates` avec le format `nom_template.html`

## Partie 0 - Initialisation de l'environnement

Nous avons suivi la méthode donné dans le pdf : 

**Étape 1 :**
+ Récupérer depuis Moodle et dézipper les fichiers :
    + djangoTP.zip .
    + pyfhn.zip qui est une bibliothèque nécessaire.
+ Sous PyCharm, ouvrir le projet djangoTP et ajouter les bibliothèques pandas,spicy et pyfhn avec python packages.
+ Lancer le serveur Web : `python manage.py runserver`.
+ Se rendre sur http://127.0.0.1:8000/. Si l’interface s’ouvre bien, vous pouvez passer à l’étape 2.



**Étape 2 :**

Pour pouvoir utiliser l’interface, il faut créer un superutilisateur qui sera sauvegardé dans la table « auth_user » de la base de données.
Pour cela, tapez la commande :
`python manage.py createsuperuser --username=joe --email=joe@example.com`
Dès lors, vous pouvez vous authentifier sur l’interface et faire des simulations.

Nous avons créé un superuser du nom de joe, adresse mail : joe@example.com

## Partie 1 - Utilisateurs

### QUESTION 1 : Mettre en place le formulaire de création d’un utilisateur, il faudra vérifier que le compte n’existe pas.

Etape 1 : création de l'URL

In [ ]:
url(r'^new_user/$', views.add_user, name='add_user')

Etape 2 : création du formulaire

In [1]:
class Creation_Profil(UserCreationForm):
    # Hérite de UserCreationForm
    class Meta:
        # Suit le modèle : User
        model = User
        # Choix des champs que nous voulons que l'utilisateur rentre pour son inscription
        fields = ["first_name", "last_name", "email", "username", "password1", "password2"]

    def email_disponible(self, email):
        """
        Méthode permettant de vérifier que l'email est disponible
        """
        liste = User.objects.filter(email=email)  # va lister tous les utilisateurs avec cette adresse mail
        if not liste:  # Si la liste est vide, l'adresse mail n'est pas prise : retourne vrai (email disponible)
            return True
        return False  # Sinon retourne Faux (email non disponible)

Nous n'avons pas besoin de faire de modification des tables de la BDD. 

Nous passons directement à l'étape 4 : création de la vue

In [ ]:
@require_http_methods(["GET", "POST"])
def add_user(request):
    """
    Fonction permettant de créer un nouvel utilisateur
    """
    # Valeurs booléennes pour le template
    email_not_available = False
    erreur = False

    # Gestion du formulaire
    if request.method == "POST":
        # Initialisation du formulaire
        form = Creation_Profil(request.POST)

        # Test si le formulaire est correctement rempli et que l'email est de format correct
        if form.is_valid() and form.cleaned_data["email"]:

            # Test si l'email est disponible (méthode créé dans le forms.py)
            if form.email_disponible(request.POST["email"]):
                # Si tout est correct enregistre le formulaire dans la BDD
                form.save()
                # Et redirige l'utilisateur vers l'accueil
                return redirect("/")
            else:
                # Sinon changement de la valeur booléenne, modifiant l'affichage
                email_not_available = True
        else:
            # Sinon changement de la valeur booléenne, modifiant l'affichage
            erreur = True
    else:
        # Affiche le formulaire
        form = Creation_Profil()
    return render(request, "new_profile.html", {"form": form, "email_not_available": email_not_available, "erreur": erreur})

Nous avons enfin écrit la page HTML, `new_profile.html` :

In [ ]:
{% extends "base.html" %}
{% block content %}
{% if email_not_available %}<p style="color:red">Un compte avec cette adresse mail existe déjà.</p>{% endif %}
{% if erreur %}<p style="color:red">Formulaire incorrect.</p>{% endif %}


<h3>Nouvel utilisateur : </h3>
<form action="{% url 'add_user' %}" method="post">
    {% csrf_token %}
    {{ form.as_p }}
    <input type="submit" value="S'inscrire" />
</form>

{% endblock %}

### QUESTION 2 : Permettre à l’utilisateur de modifier son mot de passe.

Etape 1 : création de l'URL

In [ ]:
url(r'^change_password/$', views.change_password, name='change_password')

Etape 2 : Création du formulaire

In [ ]:
class Changer_MDP(PasswordChangeForm):
    # Hérite de PasswordChangeForm
    class Meta:
        # Suit le modèle PasswordChangeForm
        model = PasswordChangeForm
        # Choix des champs : son ancien mot de passe, son nouveau mot de passe et sa confirmation
        fields = ['password1', 'password2']

Comme précédemment nous n'avons pas à modifier le modele. 

Nous passons donc étape 4 : création de la vue

In [ ]:
@login_required(login_url="/account/login/")
@require_http_methods(["GET", "POST"])
def change_password(request):
    """
    Fonction permettant de changer le mdp de l'utilisateur
    """
    # Valeur booléenne pour le template
    erreur_password = False

    # Gestion du formulaire
    if request.method == "POST":
        form = Changer_MDP(request.user, data=request.POST)

        # Test si le formulaire est valide
        if form.is_valid():
            # Si c'est le cas enregistre les modifications
            form.save()
            # Permet de garder l'utilisateur connecté
            update_session_auth_hash(request, form.user)
            # Redirige vers l'accueil
            return redirect("/")
    else:
        # Affiche le formulaire
        form = Changer_MDP(request.user)
    return render(request, "change_password.html", {"form": form, "erreur_mdp": erreur_password})

Puis de sa page HTML, `change_password.html`:

In [ ]:
{% extends "base.html" %}
{% block content %}
<h3>User: {{user.username}}</h3>
<form action="{% url 'change_password' %}" method="post">
    {% csrf_token %}
    {{ form.as_p }}
    <p><input type="submit" value="Changer mot de passe" /></p>
    <p><a href="/profile" class="btn btn-success">Retour page : Profile</a></p>
</form>

{% endblock %}

### QUESTION 3 : Permettre à l’utilisateur de supprimer son compte.

Etape 1 : création de l'URL

In [ ]:
url(r'^del_user/$', views.del_user, name='del_user')

Etape 2 : création du formulaire

In [ ]:
class Supprimer_Profil(forms.Form):
    texte_confirmation = forms.CharField(max_length=9,  # permet de fixer un nombre maximum de caractère
                                         widget=forms.TextInput(),  # fixe le <widget
                                         help_text="Si vous voulez supprimer le compte tapez : CONFIRMER")  # permet d'afficher un texte d'aide

Etape 4 : (pas de modification de la BDD) création de la vue

In [ ]:
@login_required(login_url="/account/login/")
@require_http_methods(["GET", "POST"])
def del_user(request):
    """
    Fonction permettant de changer le mdp de l'utilisateur
    """
    # Valeur booléenne pour le template
    erreur_saisie = False

    # Gestion du formulaire
    if request.method == "POST":
        form = Supprimer_Profil(request.POST)

        # Permet de tester si le formulaire est correct et que le texte entré est bien : CONFIRMER
        if form.is_valid() and request.POST["texte_confirmation"] == "CONFIRMER":
            # Récupère l'utilisateur courant
            current_user = User.objects.get(username=request.user.username)
            # Le supprime de la BDD
            current_user.delete()
            # Redirige vers l'accueil
            return redirect("/")
        else:
            erreur_saisie = True
    else:
        # Affiche le formulaire
        form = Supprimer_Profil()
    return render(request, "del_user.html", {"form": form, "erreur_saisie": erreur_saisie})

Etape 5: création de la page HTML, `del_user.html`

In [ ]:
{% extends "base.html" %}
{% block content %}
<h3>User: {{user.username}}</h3>

{% if erreur_saisie %}<p style="color:darkred">Tapez le mot CONFIRMER, attention à la casse.</p>{% endif %}

<form action="{% url 'del_user' %}" method="post">
    {% csrf_token %}
    {{ form.as_p }}
    <p><input type="submit" value="Suppression" /></p>
    <p><a href="/profile" class="btn btn-success">Retour page : Profile</a></p>
</form>

{% endblock %}

## Partie 2 - Simulations (supplémentaire)

### QUESTION 4 : Permettre à un utilisateur de partager une simulation avec un autre utilisateur.

Etape 1 : création de l'URL

In [ ]:
url(r'^(?P<object_id>[0-9]+)/share_sim/$', views.simulation_share, name='share_sim')

Etape 2 : Création du formulaire

In [ ]:
class Shared_Simulation(forms.ModelForm):
    class Meta:
        model = Partage_Simulation
        fields = ['destinataire']

Etape 3 : Modification de la BDD : création de la table `Partage_Simulation`:

In [ ]:
class Partage_Simulation(models.Model):
    # Création de la table
    
    # Stocke les clés primaires de l'expéditeur, du destinataire et de la simulation
    expediteur = models.ForeignKey(User, on_delete=models.CASCADE,  related_name="expediteur", null=True)
    destinataire = models.ForeignKey(User, on_delete=models.CASCADE, related_name="destinataire", null=True)
    simulation = models.ForeignKey(Simulation, on_delete=models.CASCADE, related_name="simulation_shared_id", null=True)

Etape 4 : Création de la vue

In [ ]:
@login_required(login_url="/account/login/")
@require_http_methods(["GET", "POST"])
def simulation_share(request, object_id):
    # Valeur booléenne pour le template
    impossible_send = False

    # Gestion du formulaire
    if request.method == "POST":
        form = Shared_Simulation(request.POST)

        # Test si le formulaire est valide
        if form.is_valid() and form.cleaned_data['destinataire']:
            expediteur = request.user
            destinataire = form.cleaned_data['destinataire']

            # Test si le destinataire a déjà reçu la simulation ou si l'utilisateur partage la simulation a lui-même
            if Partage_Simulation.objects.filter(destinataire=destinataire).exists() or expediteur == destinataire:
                # Change la valeur du booléen
                impossible_send = True
            else:
                # Récupère la simulation
                sim = Simulation.objects.get(pk=object_id)
                # L'ajoute à la BDD : Partage_Simulation et l'enregistre (.save())
                Partage_Simulation(expediteur=expediteur, destinataire=destinataire, simulation=sim).save()
                # Redirige vers la liste des simulations
                return redirect("sim_list")
    else:
        # Affiche le formulaire
        form = Shared_Simulation()
    return render(request, "simulation_share.html", {"form": form, "impossible_send": impossible_send})

Etape 5 : création de la page HTML, `simulation_share.html`:

In [ ]:
{% extends "base.html" %}
{% block content %}
<h3>Partage de simulation</h3>
<form method="post">
    {% csrf_token %}
    {% if impossible_send %}<p style="color:darkred">Vous ne pouvez pas partager deux fois la simulation à la même personne, ou vous la partager.</p>{% endif %}
    {{ form.as_p }}
    <a href="/sim_list" class="btn btn-success">Retour page simulation</a>
    <button type="submit" class="btn btn-success">Share</button>
</form>
{% endblock %}

### Question 5 : Ajouter un système de favoris sur les simulations. Les simulations favoris se placeront en haut de la liste.

Etape 1 : création de l'URL

In [ ]:
url(r'^(?P<object_id>[0-9]+)/fav_sim/$', views.simulation_fav, name='fav_sim')

Etape 3 : Modification de la BDD (nous n'avons pas de formulaire associé)

In [ ]:
class Simulation(models.Model):
    user_creator = models.ForeignKey(User, on_delete=models.PROTECT, related_name="user_id")  # Changement du nom
    date = models.DateTimeField(auto_now_add=True, auto_now=False)
    alpha = models.FloatField(validators=[MinValueValidator(0.08), MaxValueValidator(0.12)],)
    beta = models.FloatField(validators=[MinValueValidator(0.4), MaxValueValidator(0.6)],)
    gamma = models.FloatField(validators=[MinValueValidator(0.04), MaxValueValidator(0.06)],)
    delta = models.FloatField(validators=[MinValueValidator(0.008), MaxValueValidator(0.012)],)
    epsilon = models.FloatField(validators=[MinValueValidator(0.08), MaxValueValidator(0.12)],)
    is_favorite = models.BooleanField(default=False) # Ajout d'une valeur booléenne

Etape 4 : Création de la vue associée

In [ ]:
@login_required(login_url="/account/login/")
@require_http_methods(["GET", "POST"])
def simulation_fav(request, object_id):
    """
    Vue permettant de modifier la valeur du booléen : is_favorite
    """
    # Récupération de la simulation
    sim = get_object_or_404(Simulation, pk=object_id)

    # La modification va dépendre de son état actuel
    if sim.is_favorite:
        # Si la valeur est True nous la mettons à False
        sim.is_favorite = False
        # Puis nous enregistrons la modification
        sim.save()
    else:
        # Si la valeur est False nous la mettons à True
        sim.is_favorite = True
        # Puis nous enregistrons la modification
        sim.save()
    return redirect("sim_list")

Nous n'avons pas de page HTML, liée à cette option. La modification et l'affichage ne se fait que sur la page `simulation_list.html` que nous avons modifiée : 

In [ ]:
{% extends "base.html" %}
{% block content %}
{% csrf_token %}
<h2>Simulations faites:</h2>
<script type="text/javascript">
    $(document).on('click', '.confirm-delete', function(){
        ;
    })
</script>
<h3>Favoris :</h3>
<ul>
{% for sim in user_sims %}
    {% if sim.is_favorite %}
        <li>Créateur :{{sim.user_creator}}, Paramètres :[{{sim.alpha}},{{sim.beta}},{{sim.gamma}},{{sim.delta}},{{sim.epsilon}}]
            <a href="{% url 'run_sim' sim.id %}">Re-run</a>
            <a href="{% url 'delete_sim' sim.id %}">Delete</a>
            <a href="{% url 'share_sim' sim.id %}">Share</a>
            <a href="{% url 'fav_sim' sim.id %}">Unfav</a>
        </li>
    {% endif %}
{% endfor %}
</ul>

<h3>Non favoris :</h3>
<ul>
{% for sim in user_sims %}
    {% if not sim.is_favorite %}

        <li>Créateur :{{sim.user_creator}}, Paramètres :[{{sim.alpha}},{{sim.beta}},{{sim.gamma}},{{sim.delta}},{{sim.epsilon}}]
            <a href="{% url 'run_sim' sim.id %}">Re-run</a>
            <a href="{% url 'delete_sim' sim.id %}">Delete</a>
            <a href="{% url 'share_sim' sim.id %}">Share</a>
            <a href="{% url 'fav_sim' sim.id %}">Fav</a>
        </li>

    {% endif %}
{% endfor %}
</ul>

<h2>Simulations reçus:</h2>
<script type="text/javascript">
    $(document).on('click', '.confirm-delete', function(){
        ;
    })
</script>
<ul>
{% for sim in user_shared %}
        <li>Créateur :{{sim.user_creator}}, Paramètres :[{{sim.alpha}},{{sim.beta}},{{sim.gamma}},{{sim.delta}},{{sim.epsilon}}]
            <a href="{% url 'run_sim' sim.id %}">Re-run</a>
            <a href="{% url 'delete_sim' sim.id %}">Delete</a>
            <a href="{% url 'share_sim' sim.id %}">Share</a>
        </li>
{% endfor %}
{% endblock %}

## Vues que nous avons modifiées (autres que celles du dessus) : 

La vue permettant l'affichage des simulations. Nous lui avons ajouté les simulations reçues par l'utilisateur.

In [ ]:
@login_required(login_url="/account/login/")
@require_http_methods(["GET", "POST"])
def simulation_list(request):
    """
    Vue permettant de lister l'ensemble des simulations liés à un utilisateur. A savoir celles qu'il a créé
    (en favori ou non) et celles qu'on lui a partagé
    """
    # Liste des simulations qu'il a créé
    simulation_user = Simulation.objects.filter(user_creator=request.user)

    # Liste des simulations qu'on lui a partagé
    # Récupération sous forme de liste des id de simulation pour lesquelles sont id est dans la classe destinataire
    shared_simulations_id = Partage_Simulation.objects.filter(destinataire=request.user).values_list('simulation')
    shared_simulations_id = list(elt[0] for elt in shared_simulations_id)  # Récupération brute des valeurs
    # Recherche dans la table de simulation des id des simulations
    shared_simulations = Simulation.objects.filter(id__in=shared_simulations_id)

    # Renvoie vers la page avec les deux listes
    return render(request, "simulation_list.html", {"user_sims": simulation_user, "user_shared": shared_simulations})

La vue permettant la modification du profil afin d'éviter l'erreur adresse mail déjà utilisé, si l'utilisateur ne la change pas.

In [ ]:
@login_required(login_url="/account/login/")
@require_http_methods(["GET", "POST"])
def edit_profile(request):
    """
    Vue permettant d'éditer les champs : first_name; last_name et email
    """
    # Valeurs booléennes pour le template
    envoi = False
    erreur_email = False

    # Gestion du formulaire
    if request.method == "POST":
        # Initialisation du formulaire
        user_profile_form = UserProfileForm(request.POST)
        # Récupération de l'utilisateur courant
        current_user = User.objects.get(username=request.user.username)

        # Test si le formulaire est correctement rempli
        if user_profile_form.is_valid() and user_profile_form.cleaned_data["email"]:

            # Test si l'utilisateur a changé ou non son adresse mail
            if current_user.email == request.POST["email"]:
                # Mise à jour des champs de l'utilisateur courant, s'il ne change pas son adresse mail
                current_user.first_name = user_profile_form.cleaned_data["first_name"]
                current_user.last_name = user_profile_form.cleaned_data["last_name"]
                # Sauvegarde dans la base de données
                current_user.save()
                # Changement de la valeur booléenne
                envoi = True

            # Sinon teste si l'email est disponible (méthode créé dans le forms.py)
            elif user_profile_form.email_disponible(user_profile_form.cleaned_data["email"]):
                # Mise à jour des champs de l'utilisateur courant, s'il change son adresse mai
                current_user.first_name = user_profile_form.cleaned_data["first_name"]
                current_user.last_name = user_profile_form.cleaned_data["last_name"]
                current_user.email = user_profile_form.cleaned_data["email"]
                # Sauvegarde dans la base de données
                current_user.save()
                # Changement de la valeur booléenne
                envoi = True

            # Sinon change la valeur booléenne permettant un affichage dynamique de l'erreur d'adresse mail
            else:
                erreur_email = True
    else:
        # Affiche le formulaire pré-rempli
        user_profile_form = UserProfileForm(instance=request.user)
    return render(request, "edit_profile.html", {"form": user_profile_form, "envoi":envoi, "erreur_email":erreur_email})

Et le formulaire associé. 

In [ ]:
class UserProfileForm(forms.ModelForm):
    class Meta:
        model = User
        fields = ['first_name', 'last_name', 'email']

    def email_disponible(self, email):
        liste = User.objects.filter(email=email)  # va lister tous les utilisateurs avec cette adresse mail
        if not liste:  # Si la liste est vide, l'adresse mail n'est pas prise : retourne vrai (email disponible)
            return True
        return False  # Sinon retourne Faux (email non disponible)

Le formulaire associé à la création de simulation (nous lui avons enlever l'attribut user. L'utilisateur ne peut lancer des simulations que pour lui.

In [ ]:
class SimuForm(forms.ModelForm):
    class Meta:
        model = Simulation
        fields = ['alpha', 'beta', 'gamma', 'delta', 'epsilon']

Et les vues associées.

In [ ]:
@login_required(login_url="/account/login/")
@require_http_methods(["GET", "POST"])
def new_simu(request):
    """
    Construire le formulaire, soit avec les données postées, soit vide si l'utilisateur accède pour la première fois
    à la page.
    """
    form = SimuForm(request.POST, request.FILES)

    # Si le formulaire est valide
    if form.is_valid():
        params = form.cleaned_data
        print(params)
        # Création de la simulation
        newsim = Simulation.objects.create(
            user_creator=request.user,
            alpha=params["alpha"],
            beta=params["beta"],
            gamma=params["gamma"],
            delta=params["delta"],
            epsilon=params["epsilon"],
        )
        # Sauvegarde dans la base de données
        newsim.save()
        # Lance la simulation en appelant la fonction run_sim
        return run_sim(request, newsim.id)
    return render(request, "newsimu.html", locals())



@require_http_methods(["GET", "POST"])
def run_sim(request, object_id):
    """
    Effectue la simulation puis la retourne vers le template
    """
    # Récupération des informations de la simulation
    params = model_to_dict(get_object_or_404(Simulation, pk=object_id))

    # Suppression des paramètres inutiles pour le calcul
    # A savoir : username (user_creator), id de la simulation et is_favorite
    params.pop("user_creator")
    params.pop("id")
    params.pop("is_favorite")

    # Récupération des dernières informations : alpha, beta, delta, gamma, epsilon
    res = run_fhn_base(params)

    # Mise en page de la simulation
    f = plt.figure()
    plt.title("FHN Simulation")
    plt.xlabel("Time")
    plt.ylabel("Outputs")
    plot = plt.plot(res["t"], res["y"][0], res["t"], res["y"][1])
    plt.legend(["v", "w"])
    imgdata = StringIO()
    f.savefig(imgdata, format="svg")
    imgdata.seek(0)

    # data contenant le graphique
    data = imgdata.getvalue()
    return render(request, "graphic.html", {"graphic": data})

De même nous avons modifié la vue permettant la suppression d'une simulation (en fonction de si l'utilisateur en est l'auteur ou non).

In [ ]:
@login_required(login_url="/login/")
@require_http_methods(["GET", "POST"])
def simulation_delete(request, object_id):
    """
    Lié à la vue de sim_list, nous pouvons supprimer une simulation de la liste de l'utilisateur
    """
    # Récupération de la simulation
    sim = get_object_or_404(Simulation, pk=object_id)

    # La suppression de la simulation va dépendre de son auteur
    if request.user == sim.user_creator:
        # Si l'utilisateur est à l'origine de cette simulation, elle va être totalement supprimé
        sim.delete()
    else:
        # Sinon la simulation lui a été partagé et nous ne la supprimons que de la table de simulation
        # Ainsi elle sera toujours présente pour son auteur ou les autres personnes l'ayant reçu
        share_sim = Partage_Simulation.objects.filter(destinataire=request.user, simulation=object_id)
        share_sim.delete()
    return HttpResponseRedirect(reverse_lazy("sim_list"))

# Page HTML que nous avons modifiées (autres que celles de l'exercice)

Celle de login :

In [ ]:
{% extends 'base.html' %}

{% block title %}Connexion{% endblock %}

{% block content %}
    <h2>Connexion :</h2>
    <form method="post">
        {% csrf_token %}
        {{ form.as_p }}

    <p><a class="button button-primary" href="/new_user">Inscription</a></p>
    <p><button type="submit">Envoyer</button></p>
    </form>
{% endblock %}

Celle de logout

In [ ]:
{% extends "base.html" %}
{% block content %}
{% include "django_pam/accounts/_logout.html" %}
{% endblock %}

De même nous avons rajouté ces deux lignes dans le fichier `settings.py`:

In [ ]:
LOGIN_REDIRECT_URL = '/'
LOGOUT_REDIRECT_URL = '/'

Celle de la base :

In [ ]:
{% load static %}
<!DOCTYPE html>
<html lang="fr">
<body>
<header></header>
    <div class="header">
       {% block nav %}
       <a class="button button-primary" href="/">Accueil</a>
       <a class="button button-primary" href="/profile/">Profile</a>
       <a class="button button-primary" href="/change_password/">Change Password</a>
       <a class="button button-primary" href="/del_user/">Delete user</a>
       <a class="button button-primary" href="/new_user/">Add user</a>
       <a class="button button-primary" href="/sim_list/">Simulations</a>
       <a class="button button-primary" href="/newsimu/">New Simulation</a>
       {% endblock %}
   </div>

   <section id="content" class="container">
       {% block content %}
           {% if user.is_authenticated %}
                Bonjour: {{ user.username }} !
                <p><a href="{% url 'logout' %}">Déconnexion</a></p>
            {% else %}
               <p>Vous n'êtes pas connecté.</p>
                <a href="{% url 'login' %}">Connexion</a>
            {% endif %}
       {% endblock %}
   </section>

<footer class="footer"><div class="row">
    <div class="two-thirds column">Date :{% block Date %} {% now "d/m/Y" %} {% endblock %} </div>
    <div class="two-thirds column">Heure :{% block Hours %} {% now "H:i" %} {% endblock %} </div>
    <div class="one-third column">INF2 TP6</div>
    <div class="one-third column">Auteurs : XXXX YYYY</div>
  </div></footer>
</body>
</html>

Celle de la page edit_profile

In [ ]:
{% extends "base.html" %}
{% block content %}
<h3>User: {{user.username}}</h3>

<p>Renseignez vos informations personnelles ci-dessous.</p>
{%if erreur_email%}<p style="color:darkred"> Erreur email.</p>{%endif%}
<form action="{% url 'profile' %}" method="post">
    {% csrf_token %}
    {{ form.as_p }}

{%if envoi%}<p style="color:darkgreen"> Données mises à jour.</p>{%endif%}

<p><a href="/change_password" class="btn btn-success">Change Password</a></p>
<p><a href="/del_user" class="btn btn-success">Supprimer compte</a></p>
<input type="submit" value="Edit informations" /></form>
{% endblock %}

Celle de la page new_simu

In [ ]:
{% extends "base.html" %}
{% block content %}
<h3>User: {{user.username}}</h3>
<form action="{% url 'newsimu' %}" method="post" enctype="multipart/form-data">
    {% csrf_token %}
    {{ form.as_p }}
    <input type="submit" value="Run Simulation" />
</form>
{% if envoi %}Your simulation has been added to the queue !{% endif %}
{% endblock %}

Celle de la page résultat de la simulation

In [ ]:
{% extends "base.html" %}
{% block content %}
<p>
    <h3>Résultat simulation : </h3>
    {{graphic|safe}}
    <a href="{% url 'sim_list' %}" class="btn btn-dark">Retour liste</a>
</p>
{% endblock %}

### Nous avons modifé le fichier `settings.py`

In [ ]:
LOGIN_REDIRECT_URL = '/'
LOGOUT_REDIRECT_URL = '/'

AUTH_PASSWORD_VALIDATORS = [
    {
        'NAME': 'django.contrib.auth.password_validation.UserAttributeSimilarityValidator',
    },
    {
        'NAME': 'django.contrib.auth.password_validation.MinimumLengthValidator',
        'OPTIONS': {
            'min_length': 8,
        }
    },
    {
        'NAME': 'django.contrib.auth.password_validation.CommonPasswordValidator',
    },
    {
        'NAME': 'django.contrib.auth.password_validation.NumericPasswordValidator',
    },
]

LANGUAGE_CODE = 'fr-FR'

TIME_ZONE = 'Europe/Paris'